# Data Wrangling Criminal Expungement Dashboard Data

## Part 1: Setup

In [50]:
# STOP!
# Before you do anything, copy the code below. You'll need to shut down the 
# notebook, close and re-open the terminal and paste this code in *before* you call jupyter notebook.
# We need to switch to Java 8 before we can load PySpark functions. 
'''
/usr/libexec/java_home -V
export JAVA_HOME=/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home
export PATH=$JAVA_HOME/bin:$PATH
which java
'''

'\n/usr/libexec/java_home -V\nexport JAVA_HOME=/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home\nexport PATH=$JAVA_HOME/bin:$PATH\nwhich java\n'

In [51]:
# Load libraries
import os
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [52]:
# Set working directory
os.chdir('/Users/amawest/Desktop')

In [53]:
!java -version

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (AdoptOpenJDK)(build 1.8.0_292-b10)
OpenJDK 64-Bit Server VM (AdoptOpenJDK)(build 25.292-b10, mixed mode)


In [54]:
# load in all the raw data from Virginia Open Court Data
data_2010 = pd.read_csv('circuit_data/circuit_criminal_2010_anon_00.csv')
data_2011 = pd.read_csv('circuit_data/circuit_criminal_2011_anon_00.csv')
data_2012 = pd.read_csv('circuit_data/circuit_criminal_2012_anon_00.csv')
data_2013 = pd.read_csv('circuit_data/circuit_criminal_2013_anon_00.csv')
data_2014 = pd.read_csv('circuit_data/circuit_criminal_2014_anon_00.csv')
data_2015 = pd.read_csv('circuit_data/circuit_criminal_2015_anon_00.csv')
data_2016 = pd.read_csv('circuit_data/circuit_criminal_2016_anon_00.csv')
data_2017 = pd.read_csv('circuit_data/circuit_criminal_2017_anon_00.csv')
data_2018 = pd.read_csv('circuit_data/circuit_criminal_2018_anon_00.csv')
data_2019 = pd.read_csv('circuit_data/circuit_criminal_2019_anon_00.csv')

/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,27,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/amawest/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (39,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Part 2: Adding County Names Based on Zip to Match Census
- Note: This function takes a WHILE to run. I have the completed files on Github already. 

In [55]:
# Load in all data and get the county names based on zip (to match with Census)
import addfips
import zipcodes

In [56]:
# check that the functions work the way they are supposed to knowing zipcodes that you know
# this function matches county to zip
af = addfips.AddFIPS()
# test that it works based on 2 zipcodes --> counties I know
info = zipcodes.matching('22901') # my current residence (VA)
print(info[0]['county'])
info = zipcodes.matching('49046') # my hometown (MI)
print(info[0]['county'])

Albemarle County
Barry County


In [ ]:
def get_counties(file):
    """Adds the county of residence for every person with a valid zipcode."""
    
    # step 3: we need zipcodes, we we'll need to extract them from the addresses
    file[['city_state', 'zip']] = file.Address.str.rsplit(" ", n=1, expand=True,)
    file_zips = file[file['zip'].notnull()]
    
    # a few zip codes create an error because they don't have a corresponding county,
    # this should be rewritten so that the loop happening a few lines later learns
    # to just past over these zips to avoid breaking (and for when we add data later)
    file_zips = file_zips[~file_zips['zip'].str.contains('-')]
    file_zips = file_zips[file_zips['zip'].str.contains('(?<!\n)[\d]{5,6}[\-]?[\d]*')]
    file_zips = file_zips[file_zips['city_state'].str.contains('VA')]
    
    # create space for few values
    file_zips['county'] = 0
    file_zips['state']  = 'Virginia'
    
    # add in the county names 
    for i in range(0, len(file_zips)):
        # we can use try-except here because we don't need the bad zip entries to be included - they will just go
        # towards the final "missing" chunk, anyways. 
        if file_zips['county'][i] == 0:
            try:
                info = zipcodes.matching(list(file_zips['zip'])[i])
            except:
                pass
            if info == []:
                pass
            else: 
                file_zips['county'][i] = info[0]['county']
        return file_zips

In [ ]:
data_2010_zips = get_counties(data_2010)
data_2011_zips = get_counties(data_2011)
data_2012_zips = get_counties(data_2012)
data_2013_zips = get_counties(data_2013)
data_2014_zips = get_counties(data_2014)
data_2015_zips = get_counties(data_2015)
data_2016_zips = get_counties(data_2016)
data_2017_zips = get_counties(data_2017)
data_2018_zips = get_counties(data_2018)
data_2019_zips = get_counties(data_2019)

In [ ]:
# already saved, so commented out so they are not overwritten
'''
data_2010_zips.to_csv('circuit_data/zips_circuit_criminal_2010.csv')
data_2011_zips.to_csv('circuit_data/zips_circuit_criminal_2011.csv')
data_2012_zips.to_csv('circuit_data/zips_circuit_criminal_2012.csv')
data_2013_zips.to_csv('circuit_data/zips_circuit_criminal_2013.csv')
data_2014_zips.to_csv('circuit_data/zips_circuit_criminal_2014.csv')
data_2015_zips.to_csv('circuit_data/zips_circuit_criminal_2015.csv')
data_2016_zips.to_csv('circuit_data/zips_circuit_criminal_2016.csv')
data_2017_zips.to_csv('circuit_data/zips_circuit_criminal_2017.csv')
data_2018_zips.to_csv('circuit_data/zips_circuit_criminal_2018.csv')
data_2019_zips.to_csv('circuit_data/zips_circuit_criminal_2019.csv')
'''

## Part 3: Aggregate Data

In [104]:
# Read in files
# note that we only keep DISTINCT (i.e. unique) person IDs from here on out
for i in range(0,10):
    infile = (f'circuit_data/zips_circuit_criminal_201{i}.csv')
    spark = SparkSession \
        .builder \
        .appName("Racial Disparity") \
        .getOrCreate()
    circuit = spark.read.csv(infile, inferSchema = True, header = True)
    circuit.createOrReplaceTempView("data")
    sqlDF = spark.sql(
        
        # currently set to look at marijuana code sections only
        # but just take out codesections to see everything
        '''
        SELECT COUNT(DISTINCT(person_id)) as Count, Race, Sex, fips FROM data
                   WHERE (CodeSection LIKE '18.2-248%' 
                   OR CodeSection LIKE '18.2-250%'
                   OR CodeSection LIKE '18.2-255%')
                   GROUP BY Race, fips, Sex
                   ORDER BY Count DESC
                   '''
    )
    data = sqlDF.toPandas()
    data['Race and Sex'] = data['Race'] + ' - ' + data['Sex']
    data = data[['fips', 'Race and Sex', 'Count']]
    df_p = data.pivot_table(index=['fips'], columns=['Race and Sex'], values='Count', aggfunc=np.sum)
    data_1 = df_p.fillna(0)
    data_1.to_csv(f'Data/201{i}_finished.csv')

# NOTE: Add/remove as desired to change the type of query
# Marijuana
# WHERE (CodeSection LIKE '18.2-248%' 
# OR CodeSection LIKE '18.2-250%'
# OR CodeSection LIKE '18.2-255%')

# Felonys
# AND (ChargeType == 'Felony')

In [105]:
#pd.read_csv('2019_finished.csv').columns

In [106]:
#pd.read_csv('2018_finished.csv').columns

In [107]:
# Some code to rename 2019 to match 2010-2018 column labels (currently not using)
x = pd.read_csv('2019_finished.csv')
new_cols = {'American Indian Or Alaskan Native - Female':         'American Indian - Female',
            'American Indian Or Alaskan Native - Male':           'American Indian - Male',
            'Asian Or Pacific Islander - Female':                 'Asian Or Pacific Islander - Female',
            'Asian Or Pacific Islander - Male':                   'Asian Or Pacific Islander - Male',
            'Black - Female':                                     'Black (Non-Hispanic) - Female',
            'Black - Male':                                       'Black (Non-Hispanic) - Male',
            'Hispanic - Female':                                  'Hispanic - Female',
            'Hispanic - Male':                                    'Hispanic - Male',
            'White - Female':                                     'White Caucasian (Non-Hispanic) - Female',
            'White - Male':                                       'White Caucasian (Non-Hispanic) - Male',
            'Other (Includes Not Applicable, Unknown) - Female':  'Other (Includes Not Applicable, Unknown) - Female',
            'Other (Includes Not Applicable, Unknown) - Male':    'Other (Includes Not Applicable, Unknown) - Male',
           }
x.rename(columns=new_cols,
          inplace=True)
x.to_csv('Data/2019_finished.csv', index=None)

In [108]:
'''
# Note: make sure to reset the year each time
infile = 'Data/census_data_fips.csv'
spark = SparkSession \
    .builder \
    .appName("Census Data") \
    .getOrCreate()
circuit = spark.read.csv(infile, inferSchema=True, header = True)
circuit.createOrReplaceTempView("census_data")
sqlDF = spark.sql('SELECT * \
                   FROM census_data \
                   WHERE Year = 2019')
census = sqlDF.toPandas()
'''

'\n# Note: make sure to reset the year each time\ninfile = \'Data/census_data_fips.csv\'\nspark = SparkSession     .builder     .appName("Census Data")     .getOrCreate()\ncircuit = spark.read.csv(infile, inferSchema=True, header = True)\ncircuit.createOrReplaceTempView("census_data")\nsqlDF = spark.sql(\'SELECT *                    FROM census_data                    WHERE Year = 2019\')\ncensus = sqlDF.toPandas()\n'

In [109]:
for i in range(0,10):
    # Note: make sure to reset the year each time
    infile = 'Data/census_data_fips.csv'
    spark = SparkSession \
    .builder \
    .appName("Census Data") \
    .getOrCreate()
    circuit = spark.read.csv(infile, inferSchema=True, header = True)
    circuit.createOrReplaceTempView("census_data")
    sqlDF = spark.sql(f'SELECT * \
                   FROM census_data \
                   WHERE Year = 201{i}')
    census = sqlDF.toPandas()
    
    #--------------------#
    data = pd.read_csv(f'Data/201{i}_finished.csv')
    #--------------------#
    data['Total_Crimes'] = 0
    data['Total_Crimes'] = data.sum(axis=1) 
    data.head(2)
    #del census['Year']
    census.head(2)
    result = pd.merge(census,
                  data,
                  on='fips', 
                  how='left')
    result.head(2)
    data.columns
    result['Percent_Overall']          = (result['Total_Crimes']   / result['Total'])*100
    result['Percent_White_Male']       = (result['White Caucasian (Non-Hispanic) - Male']   / result['White_Male'])*100
    result['Percent_White_Female']     = (result['White Caucasian (Non-Hispanic) - Female'] / result['White_Female'])*100
    result['Percent_Black_Male']       = (result['Black (Non-Hispanic) - Male']             / result['Black_Male'])*100
    result['Percent_Black_Female']     = (result['Black (Non-Hispanic) - Female']           / result['Black_Female'])*100
    result['Percent_Nat_Amer_Male']    = (result['American Indian - Male']                  / result['Native_Amer_Male'])*100
    #result['Percent_Nat_Amer_Female']  = (result['American Indian - Female']                / result['Native_Amer_Female'])*100
    result['Percent_Hispanic_Male']    = (result['Hispanic - Male']                         / result['Hispanic_Male'])*100
    result['Percent_Hispanic_Female']  = (result['Hispanic - Female']                       / result['Hispanic_Female'])*100
    result['Percent_Asian_Pac_Male']   = (result['Asian Or Pacific Islander - Male']        / result['Asian_Male'])*100
    result['Percent_Asian_Pac_Female'] = (result['Asian Or Pacific Islander - Female']      / result['Asian_Female'])*100
    
    result['Percent_Overall']            = round(result['Percent_Overall'],3)
    result['Disparity_White_Male']       = round(result['Percent_White_Male'] - result['Percent_Overall'],3)
    result['Disparity_White_Female']     = round(result['Percent_White_Female'] - result['Percent_Overall'],3)
    result['Disparity_Black_Male']       = round(result['Percent_Black_Male'] - result['Percent_Overall'],3)
    result['Disparity_Black_Female']     = round(result['Percent_Black_Female'] - result['Percent_Overall'],3)
    result['Disparity_Nat_Amer_Male']    = round(result['Percent_Nat_Amer_Male'] - result['Percent_Overall'],3)
    #result['Disparity_Nat_Amer_Female']  = round(result['Percent_Nat_Amer_Female'] - result['Percent_Overall'],3)
    result['Disparity_Hispanic_Male']    = round(result['Percent_Hispanic_Male'] - result['Percent_Overall'],3)
    result['Disparity_Hispanic_Female']  = round(result['Percent_Hispanic_Female'] - result['Percent_Overall'],3)
    result['Disparity_Asian_Pac_Male']   = round(result['Percent_Asian_Pac_Male'] - result['Percent_Overall'],3)
    result['Disparity_Asian_Pac_Female'] = round(result['Percent_Asian_Pac_Female'] - result['Percent_Overall'],3)
    
    result = result[['fips', 
                     'CTYNAME',
                     'Percent_Overall',
                     'Disparity_White_Male',
                     'Disparity_White_Female',
                     'Disparity_Black_Male',
                     'Disparity_Black_Female', 
                     'Disparity_Nat_Amer_Male',
                     #'Disparity_Nat_Amer_Female', 
                     'Disparity_Hispanic_Male',
                     'Disparity_Hispanic_Female',
                     'Disparity_Asian_Pac_Male',
                     'Disparity_Asian_Pac_Female'
        ]]
    result
    #--------------------#
    result.to_csv(f'Data/Final/201{i}_disparities.csv',index=None)
    #--------------------#

## Part 4: Put It All Together & Finalize

In [110]:
x_2010 = pd.read_csv('Data/Final/2010_disparities.csv')
x_2011 = pd.read_csv('Data/Final/2011_disparities.csv')
x_2012 = pd.read_csv('Data/Final/2012_disparities.csv')
x_2013 = pd.read_csv('Data/Final/2013_disparities.csv')
x_2014 = pd.read_csv('Data/Final/2014_disparities.csv')
x_2015 = pd.read_csv('Data/Final/2015_disparities.csv')
x_2016 = pd.read_csv('Data/Final/2016_disparities.csv')
x_2017 = pd.read_csv('Data/Final/2017_disparities.csv')
x_2018 = pd.read_csv('Data/Final/2018_disparities.csv')
x_2019 = pd.read_csv('Data/Final/2019_disparities.csv')

In [111]:
x_2010['year'] = 2010
x_2011['year'] = 2011
x_2012['year'] = 2012
x_2013['year'] = 2013
x_2014['year'] = 2014
x_2015['year'] = 2015
x_2016['year'] = 2016
x_2017['year'] = 2017
x_2018['year'] = 2018
x_2019['year'] = 2019

In [112]:
x_2010.to_csv('Data/Final/2010_disparities.csv')
x_2011.to_csv('Data/Final/2011_disparities.csv')
x_2012.to_csv('Data/Final/2012_disparities.csv')
x_2013.to_csv('Data/Final/2013_disparities.csv')
x_2014.to_csv('Data/Final/2014_disparities.csv')
x_2015.to_csv('Data/Final/2015_disparities.csv')
x_2016.to_csv('Data/Final/2016_disparities.csv')
x_2017.to_csv('Data/Final/2017_disparities.csv')
x_2018.to_csv('Data/Final/2018_disparities.csv')
x_2019.to_csv('Data/Final/2019_disparities.csv')

In [113]:
files = ['Data/Final/2010_disparities.csv',
         'Data/Final/2011_disparities.csv',
         'Data/Final/2012_disparities.csv',
         'Data/Final/2013_disparities.csv',
         'Data/Final/2014_disparities.csv',
         'Data/Final/2015_disparities.csv',
         'Data/Final/2016_disparities.csv',
         'Data/Final/2017_disparities.csv',
         'Data/Final/2018_disparities.csv',
         'Data/Final/2019_disparities.csv'
         ]

In [114]:
dfs = [pd.read_csv(f, sep=",") for f in files]
all_files = pd.concat(dfs,ignore_index=True)

In [115]:
del all_files['Unnamed: 0']

In [116]:
all_files.head()

,fips,CTYNAME,Percent_Overall,Disparity_White_Male,Disparity_White_Female,Disparity_Black_Male,Disparity_Black_Female,Disparity_Nat_Amer_Male,Disparity_Hispanic_Male,Disparity_Hispanic_Female,Disparity_Asian_Pac_Male,Disparity_Asian_Pac_Female,year
0,1,Accomack County,0.136,-0.039,-0.127,0.463,-0.017,-0.136,-0.136,-0.136,-0.136,-0.136,2010
1,3,Albemarle County,0.061,-0.010,-0.052,0.450,0.017,-0.061,0.145,-0.061,-0.061,-0.061,2010
2,5,Alleghany County,0.704,-0.029,-0.368,6.607,0.640,-0.704,-0.704,-0.704,-0.704,-0.704,2010
3,7,Amelia County,0.251,-0.062,-0.188,0.632,-0.187,-0.251,-0.251,-0.251,-0.251,-0.251,2010
4,9,Amherst County,0.219,-0.018,-0.128,0.575,-0.156,-0.219,-0.219,-0.219,-0.219,-0.219,2010


In [49]:
all_files.to_csv('all_files_1.csv')